In [1]:
from google.colab import files
import pandas as pd
import sqlite3

uploaded = files.upload()

Saving HR-Employee-Attrition.csv to HR-Employee-Attrition.csv


In [2]:
df = pd.read_csv("HR-Employee-Attrition.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
# Create an in-memory SQLite database  (if we want to use sql ...then this step is necessary.)
conn = sqlite3.connect(":memory:") # conn is variable.

# Load the DataFrame into the SQLite database
df.to_sql("employees", conn, index=False, if_exists='replace') # to_sql method : performs the action of writing the DataFrame's data to a SQL database.

1470

In [4]:
# Step 3: Basic SQL Query - Total Employees by Department
#  GROUP BY Department, the database conceptually organizes these employees into separate buckets based on their department
# The COUNT(*) function then goes into each of these individual buckets (groups) and counts how many employees (rows) are inside.

query1 = '''
SELECT
 Department,
 COUNT(*) AS TotalEmployees
 FROM employees
 GROUP BY Department
'''
pd.read_sql_query(query1, conn)

,Department,TotalEmployees
0,Human Resources,63
1,Research & Development,961
2,Sales,446


In [5]:
# Step 4: Attrition Rate by Department
# count (*) counts the total no. of employees.
# AS---> is a alias , variable(temporary)
query2 = '''
SELECT
  Department,
  SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS AttritionRate
FROM employees
GROUP BY Department
'''
pd.read_sql_query(query2, conn)

,Department,AttritionRate
0,Human Resources,19.047619
1,Research & Development,13.839750
2,Sales,20.627803


In [6]:
# Step 5: Gender Diversity by Job Role
query3 = '''
SELECT
  JobRole,
  Gender,
  COUNT(*) AS Count
FROM employees
GROUP BY JobRole, Gender
'''
pd.read_sql_query(query3, conn)

,JobRole,Gender,Count
0,Healthcare Representative,Female,51
1,Healthcare Representative,Male,80
2,Human Resources,Female,16
3,Human Resources,Male,36
4,Laboratory Technician,Female,85
5,Laboratory Technician,Male,174
6,Manager,Female,47
7,Manager,Male,55
8,Manufacturing Director,Female,72
9,Manufacturing Director,Male,73


In [7]:
# Step 6: Average Monthly Income by Department
# ROUND(..., 2): This is a SQL function that rounds a number to a specified number of decimal places.
query4 = '''
SELECT
  Department,
  ROUND(AVG(MonthlyIncome), 2) AS AvgIncome
FROM employees
GROUP BY Department
'''
pd.read_sql_query(query4, conn)

,Department,AvgIncome
0,Human Resources,6654.51
1,Research & Development,6281.25
2,Sales,6959.17


In [8]:
# Step 7: Export Cleaned Data to CSV for Power BI Dashboard
# (Optional: clean column names / drop irrelevant columns before export)
df.to_csv("cleaned_employee_data.csv", index=False)
files.download("cleaned_employee_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>